<a href="https://colab.research.google.com/github/veronica1013/project_dsp/blob/main/Moringa_Data_Science_Prep_W2_Independent_Project_2021_09_Veronica_Isiaho_SQL_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load SQL Extension 

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite://

'Connected: @None'

# Import the CSV and Pandas

In [ ]:
import csv

In [ ]:
import pandas as pd

In [ ]:
# Go to Colab drive, in the relevant file folder and
#Click on the three dots to copy the file path for Grand Electors by State and Population by State then write below:

GrandElectors_By_State = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Week_2_Independent_Project/GrandElectors_by_state.csv')
Population_By_State = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Week_2_Independent_Project/Population_by_state.csv')

In [ ]:
%sql DROP TABLE if EXISTS GrandElectors_By_State;
%sql PERSIST GrandElectors_By_State;

 * sqlite://
Done.
 * sqlite://


'Persisted grandelectors_by_state'

In [ ]:
%sql DROP TABLE if EXISTS Population_By_State;
%sql PERSIST Population_By_State;

 * sqlite://
Done.
 * sqlite://


'Persisted population_by_state'

# Cast state names from lower to upper case in grand electors tables and create a new table called GrandElectorsPopulation 

In [ ]:
# 1. Cast grand electors tables' state names from lower case to upper case.
# 2. Join Population table to grand electors table by state name.
# 3. Create a new table and name it GrandElectorsPopulation.

%%sql
CREATE TABLE GrandElectorsPopulation AS SELECT table_a.State, table_a.GrandElectors, table_b.Population FROM (
    SELECT UPPER(State) AS State, GrandElectors
    FROM GrandElectors_By_State
) AS table_a
JOIN Population_By_State AS table_b ON table_b.State = table_a.State;

 * sqlite://
Done.


[]

In [ ]:
# Select 5 rows from the GrandElectorsPopulation table

%%sql
SELECT * FROM GrandElectorsPopulation LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818


# Update the name of District of Columbia to DC

In [ ]:
# 1. Update the name of District of Columbia to DC
# 2. Select from the GrandElectorsPopulation table where state is DC

%%sql
UPDATE GrandElectorsPopulation SET State = 'DC' WHERE State = 'DISTRICT OF COLUMBIA';
SELECT * FROM GrandElectorsPopulation WHERE State = 'DC';

 * sqlite://
1 rows affected.
Done.


State,GrandElectors,Population
DC,3,672228


In [ ]:
# Add float column to the GrandElectorsPopulation table

%%sql
ALTER TABLE GrandElectorsPopulation ADD COLUMN GPRatio float(10);

 * sqlite://
Done.


[]

In [ ]:
# Update GPRatio column with computed ratio between grand electors and population

%%sql
UPDATE GrandElectorsPopulation SET GPRatio = Population/GrandElectors;

 * sqlite://
51 rows affected.


[]

In [ ]:
# Order the states by decreasing GPRatio column

%%sql
SELECT * FROM GrandElectorsPopulation ORDER BY GPRatio DESC LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,GPRatio
TEXAS,38,27469114,722871.0
CALIFORNIA,55,39144818,711723.0
FLORIDA,29,20271272,699009.0
NEW YORK,29,19795791,682613.0
NORTH CAROLINA,15,10042802,669520.0


In [ ]:
# Compute the running total of Grand Electors in that sorted list

%%sql
SELECT
  SUM(GrandElectors)  
FROM
  GrandElectorsPopulation;

 * sqlite://
Done.


SUM(GrandElectors)
538


In [ ]:
# To compute the half of the total of Grand Electors overall (in the whole country)

%%sql
SELECT
  SUM(GrandElectors)/2  
FROM
  GrandElectorsPopulation;



 * sqlite://
Done.


SUM(GrandElectors)/2
269


In [ ]:
# To filter sorted list of states in order to keep only the (top) ones
# To reach the computed threshold of 269

%%sql
SELECT * FROM GrandElectorsPopulation ORDER BY GrandElectors DESC LIMIT 12 


 * sqlite://
Done.


State,GrandElectors,Population,GPRatio
CALIFORNIA,55,39144818,711723.0
TEXAS,38,27469114,722871.0
FLORIDA,29,20271272,699009.0
NEW YORK,29,19795791,682613.0
ILLINOIS,20,12859995,642999.0
PENNSYLVANIA,20,12802503,640125.0
OHIO,18,11613423,645190.0
GEORGIA,16,10214860,638428.0
MICHIGAN,16,9922576,620161.0
NORTH CAROLINA,15,10042802,669520.0


# Recommendation

In [ ]:
# From the above workings, it is evident enough that the top 12 states with highest number of electors are the point of target for a candidate to win the US elections by garnering 283 votes from grand electors